<a href="https://colab.research.google.com/github/adelacvg/detail_tts/blob/master/demo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!git clone https://github.com/adelacvg/detail_tts.git

Cloning into 'detail_tts'...
remote: Enumerating objects: 204, done.
remote: Counting objects: 100% (204/204), done.
remote: Compressing objects: 100% (129/129), done.
remote: Total 204 (delta 76), reused 198 (delta 70), pack-reused 0
Receiving objects: 100% (204/204), 2.60 MiB | 7.02 MiB/s, done.
Resolving deltas: 100% (76/76), done.


In [2]:
%cd detail_tts

/content/detail_tts


In [3]:
!git lfs install
!git clone https://huggingface.co/adelacvg/Detail

Updated git hooks.
Git LFS initialized.
Cloning into 'Detail'...
remote: Enumerating objects: 11, done.
remote: Counting objects: 100% (7/7), done.
remote: Compressing objects: 100% (7/7), done.
remote: Total 11 (delta 1), reused 0 (delta 0), pack-reused 4 (from 1)
Unpacking objects: 100% (11/11), 3.32 KiB | 378.00 KiB/s, done.
Filtering content: 100% (3/3), 3.74 GiB | 54.40 MiB/s, done.


In [4]:
import locale
locale.getpreferredencoding = lambda: "UTF-8"
!pip install pypinyin einops omegaconf==2.0.6 k_diffusion

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 18.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.2/43.2 kB 4.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 309.4/309.4 kB 34.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 38.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 825.4/825.4 kB 52.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.2/61.2 kB 10.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 68.2 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  

In [5]:
from pypinyin import lazy_pinyin, Style
import torch
import json

MODELS = {
    'vqvae.pth':'Detail/model-575.pt',
}

device = 'cuda:0'
from bpe_tokenizers.voice_tokenizer import VoiceBpeTokenizer
import torch.nn.functional as F

cond_audio = '1.wav'
text = "大家好，今天来点大家想看的东西。"
# text = "霞浦县衙城镇乌旗瓦窑村水位猛涨。"
# text = '高德官方网站，拥有全面、精准的地点信息，公交驾车路线规划，特色语音导航，商家团购、优惠信息。'
# text = '四是四，十是十，十四是十四，四十是四十。'
# text = '八百标兵奔北坡，炮兵并排北边跑。炮兵怕把标兵碰，标兵怕碰炮兵炮。'
# text = '黑化肥发灰，灰化肥发黑。黑化肥挥发会发灰；灰化肥挥发会发黑。'
# text = '先帝创业未半而中道崩殂，今天下三分，益州疲弊，此诚危急存亡之秋也。'
text = ' '.join(lazy_pinyin(text, style=Style.TONE3, neutral_tone_with_five=True))
text = ' '+text+' '
tokenizer = VoiceBpeTokenizer('bpe_tokenizers/zh_tokenizer.json')
text_tokens = torch.IntTensor(tokenizer.encode(text)).unsqueeze(0).to(device)
text_tokens = F.pad(text_tokens, (0, 1))  # This may not be necessary.
text_tokens = text_tokens.to(device)
print(text)
print(text_tokens)
from prepare.load_infer import load_model
import torchaudio
from vqvae.utils.data_utils import spectrogram_torch,HParams
# device = 'gpu:0'
vqvae = load_model('vqvae', MODELS['vqvae.pth'], 'vqvae/configs/config.json', device)
audio,sr = torchaudio.load(cond_audio)
if audio.shape[0]>1:
    audio = audio[0].unsqueeze(0)
audio = torchaudio.transforms.Resample(sr,44100)(audio)
hps = HParams(**json.load(open('vqvae/configs/config.json')))
spec = spectrogram_torch(audio, hps.data.filter_length,
    hps.data.hop_length, hps.data.win_length, center=False).to(device)
spec_lengths = torch.LongTensor([spec.shape[-1]]).to(device)
text_lengths = torch.LongTensor([text_tokens.shape[-1]])
with torch.no_grad():
    wav = vqvae.infer(text_tokens, text_lengths, spec, spec_lengths)
torchaudio.save('gen.wav', wav.squeeze(0).cpu(), 44100)

 da4 jia1 hao3 ， jin1 tian1 lai2 dian3 da4 jia1 xiang3 kan4 de5 dong1 xi1 。 
tensor([[  2, 161,   2, 155,   2,  16,  87,   2,  43,   2, 224,   2, 171,  71,
           2, 182,   2, 188,   2, 161,   2, 155,   2,  62,  92,   2,  19,  63,
           2,  65,   2,  12,  84,   2, 228,   2,  39,   2,   0]],
       device='cuda:0', dtype=torch.int32)


/usr/local/lib/python3.10/dist-packages/torch/nn/utils/weight_norm.py:28: UserWarning: torch.nn.utils.weight_norm is deprecated in favor of torch.nn.utils.parametrizations.weight_norm.
  warnings.warn("torch.nn.utils.weight_norm is deprecated in favor of torch.nn.utils.parametrizations.weight_norm.")
/usr/local/lib/python3.10/dist-packages/transformers/configuration_utils.py:364: UserWarning: Passing `gradient_checkpointing` to a config initialization is deprecated and will be removed in v5 Transformers. Using `model.gradient_checkpointing_enable()` instead, or if you are using the `Trainer` API, pass `gradient_checkpointing=True` in your `TrainingArguments`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torch/nn/modules/conv.py:306: UserWarning: Plan failed with a cudnnException: CUDNN_BACKEND_EXECUTION_PLAN_DESCRIPTOR: cudnnFinalize Descriptor Failed cudnn_status: CUDNN_STATUS_NOT_SUPPORTED (Triggered internally at ../aten/src/ATen/native/cudnn/Conv_v8.cpp:919.)
  return F

In [6]:
from IPython.display import Audio
wav = wav.detach().cpu()
Audio(wav[0],rate=44100)